## Introduction
### RNNs types
- Many to Many
    - Sequence labelling, Machine translation. But for sequence labelling, each time step has a output.
- Many to One
    - Sentimental analysis.
- One to Many
    - Language generation.

## RNNs

$x_t$ is the **input** at time step $t$. For example, $x_1$ is one-hot vector for the second word. 

$h_t$ is the **hidden state** (i.e., **memory**) at time step $t$, $h_t=f(Wx_t+Uh_{t-1})$. $f$ is `tanh` or `ReLU`. $h_{-1}$ is typically all zeros. But there are tricks that initializing hidden state as [**trainable variables**](http://ruder.io/deep-learning-nlp-best-practices/).

$o_t$ is the **output** at time step $t$. For example, the next predicted word is $o_{t}=softmax(Vh_{t})$.

- RNNs **shares the same parameters** (i.e., $U, V, W$) across all time steps.

- The momory in LSTMs are called **cell**.
- For traditional NN, the inputs are independent. For RNNs, the inputs are sequential.
- Vanilla RNNs with BPTT have difficulities to learning long-term dependencies due to **gradients vanishing / exploding** problems. **NaN error** is signal of **gradient exploding** problem.
- Because the parameters are shared by all time steps in the network, the gradient at each output depends not only on the calculations of the current time step, but also the previous time steps. For example, in order to calculate the gradient at t=4 we would need to backpropagate 3 steps and sum up the gradients. This is called **Backpropagation Through Time (BPTT)**. 
- For bi-directional RNNs, $o_t=f(h_{leftT} + h_{rightT})$

## Language modeling
- Goal: predict the probability of the given sentence as:
$$p(w_1, w_2, ..., w_n) = \prod_{i=1}^{n}p(w_i|w_1, ..., w_{i-1})$$

- Model: let `vocabulary_size` = 8000, `hidden_layer_size` = 100, then 
$$x_t \in R^{8000}$$
$$h_t \in R^{100}$$
$$W \in R^{100*8000}$$
$$U \in R^{100*100}$$
$$V \in R^{8000*100}$$
$$o_t \in R^{8000}$$

- Data: $o_t=x_{t+1}$. For example, one training example maybe likes $x$=[0, 342, 452, 1034], and $y$=[342, 452, 1034, 1], where `0` denotes the start token and `1` denotes the end token. **START TOKEN** and **END TOKEN** is recommended, **UNKOWN TOKEN** and **PAD** should also be included.

- Types
    - **Word level**
        - Most common
    - **Character level**
        - Not predict **UNK** token
        - **Longer** than word level model, hard to train

## LSTM (Long short-term memory)
- **Vanishing gradients** prevent vanilla RNNs learn **long-term dependency**.

- LSTMs use **gate** to combat vanishing gradients, thus enable to learn long-term dependency.

- **Peephole connection** : $c_{t-1}$ is as an input to **input gate** and **forget gate**, $c_{t}$ is as an input to **output gate**.

- Model structure
    - Input gate $$i_t=\sigma (W^ix_t+U^ih_{t-1}+b^i)$$

    - Forget gate $$f_t=\sigma (W^fx_t+U^fh_{t-1}+b^f)$$

    - Output gate $$o_t=\sigma (W^ox_t+U^oh_{t-1}+b^o)$$

    - New memory cell (same as in vanilla RNNs) $$c=tanh(W^cx_t+U^ch_{t-1}+b^c)$$

    - Final memory cell $$c_t=f_t*c_{t-1}+i_t*c$$

    - Hidden state $$h_t=o_t*tanh(c_t)$$ 
    
    where $\sigma$ is `sigmoid` function, and $*$ is element-wise multiplication.
- Tensorflow initializes biases of **forget gate to a vector full of 1s** to prevents forgetting everything at the begining of training.

## GRU (Gated Recurrent Units)
- GRU has fewer parameters compared with LSTM.
- Model structure
    - Update gate $$z_t=\sigma (W^zx_t+U^zh_{t-1})$$
    
    - Reset gate $$r_t=\sigma (W^rx_t+U^rh_{t-1})$$
    
    - New memory $$h = tanh(W^hx_t+U^h(r_t*h_{t-1}))$$
    
    - Hidden state $$h_t=z_t*h_{t-1}+(1-z_t)*h$$